<a href="https://colab.research.google.com/github/wllgrnt/keras-examples/blob/master/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8

## Generative Deep Learning


In [0]:
import keras
import numpy as np
import random
import sys

## Text Generation with LSTM

Here we use RNNs to generate sequence data, specifically for the case of text data. We train a network to predict the next token, given the previous tokens - then we feed it an initial string of text and let it go. We choose the next character not using greedy sampling (choosing the most likely token) but stochastically - we control the amount of stochasticity using the softmax temperature.

In [0]:
# Reweight a probablity distribution to a different temperature
def reweight_distro(original_distro, temperature=0.5):
  distro = np.log(original_distro) / temperature
  distro = np.exp(distro)
  return distro / np.sum(distro)


### Character-level LSTM text generation

Here we train on Nietzsche.


In [4]:
path = keras.utils.get_file("nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
with open(path) as flines:
  text = flines.read().lower()

print(f"Corpus length: {len(text)}")

Corpus length: 600893


In [6]:
# Extract partially overlapping sequences of length `maxlen`, one-hot encode, and pack them into
# a 3D Numpy array `x` of shape `(sequences, maxlen, unique_characters)`. `y` will contain
# the corresponding targets - the one-hot-encoded next character in each sequence

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0,len(text)-maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
  
print(f"Number of sequences: {len(sentences)}")

chars = sorted(list(set(text)))
print(f"Unique characters: {len(chars)}")
# Dict mapping characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Binary arrays
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i,t,char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57


In [0]:
# The network is a single LSTM layer + a Dense classifier and softmax
# (we could also use 1d convnets)

model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(keras.layers.Dense(len(chars), activation="softmax"))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [0]:
# Draw from the model a probability distro for the next character
# Reweight to a given temperature
# Sample the next character
# Add the new character

def sample(preds, temperature=1.0):
  """
  Sample the next character given the model's predictions
  """
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

In [16]:
for epoch in range(1,20):
  print(f"Epoch {epoch}")
  model.fit(x,y, batch_size=128, epochs=1)
  # Select a text seed at random
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index:start_index+maxlen]
  print(f"--- Generating with seed: '{generated_text}'")
  
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    generated_text = text[start_index:start_index+maxlen]
    print(f"------ temperature: {temperature}")
    print(generated_text, end="")
    # Generate 400 characters, starting from the seed text
    for i in range(400):
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1
        
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      
      generated_text += next_char
      generated_text = generated_text[1:]
      
      print(next_char, end="")
    print()
  print()

Epoch 1
Epoch 1/1
200278/200278 [==============================] - 258s 1ms/step - loss: 1.9971
--- Generating with seed: 's not only a complex of sensation and
thinking, but it is ab'
------ temperature: 0.2
s not only a complex of sensation and
thinking, but it is able the german the servician and the spiritions of the self the self conditions of the self and the spirit and every a develong the stander and the german the stard the german the self and the servitual of the man to the self the serveration of the self has the spirit and the german the self and the german it is the german which the such a start of the self one and the self intility of the self and
------ temperature: 0.5
s not only a complex of sensation and
thinking, but it is able of a mone and the selfitions there are the seaps has as a difficulty and dount and readed and it is to their its inspiritions of a were all preself--what how a everything of the i dears it is the german the spirit and everything their the suns

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


ntaim dreverous nature,
ssoled by a even for it, alsedss. by the verience. woman rechological furning "frobest to weaugh,
------ temperature: 1.2
gical ceremony
whereby a demon is constrained to move the boluric firtor realf forms of burdee. fre?che--grayed, the nature, spirity" are
sucefuc. never aamine before mindardskes, woman. long philosophical pared way way, surtys.=-within illiny, begeat as
there is more medeame in the "present tempula,, moyesness? endience.. germans ou) hadd; and awaiti, diolatment to menteal is petes
in ofdennez, andy--he is our bichest and beegest--nesseprors, said, h"-tis

Epoch 9
Epoch 1/1
200278/200278 [==============================] - 261s 1ms/step - loss: 1.4177
--- Generating with seed: 'h we now call
life and experience--is a gradual evolution, i'
------ temperature: 0.2
h we now call
life and experience--is a gradual evolution, in the strength the philosophy the philosophy the strength of the philosophy the strength in the sense of the strength the s

## DeepDream